In [1]:
import networkx as nx
import numpy as np
import torch
import torch.nn.functional as F
from copy import deepcopy
from scipy.sparse import coo_matrix
from sklearn.metrics import roc_auc_score
import itertools
import dgl
from dgl.nn import SAGEConv
import dgl.function as fn
import json

In [2]:
import sys
sys.path.append('../src')
sys.path.append('..')

import src.synthetic as synthetic
import src.transform as transform

In [3]:
def create_train_test_split_edge(data):
    # Create a list of positive and negative edges
    u, v = data.edges()
    u, v = u.numpy(), v.numpy()
    edge_index = np.array((u, v))
    adj = coo_matrix((np.ones(data.num_edges()), edge_index))
    adj_neg = 1 - adj.todense() - np.eye(data.num_nodes())
    neg_u, neg_v = np.where(adj_neg != 0)

    # Create train/test edge split
    test_size = int(np.floor(data.num_edges() * 0.1))
    eids = np.random.permutation(np.arange(data.num_edges())) # Create an array of 'edge IDs'

    train_pos_u, train_pos_v = edge_index[:, eids[test_size:]]
    test_pos_u, test_pos_v   = edge_index[:, eids[:test_size]]

    # Sample an equal amount of negative edges from  the graph, split into train/test
    neg_eids = np.random.choice(len(neg_u), data.num_edges())
    test_neg_u, test_neg_v = (
        neg_u[neg_eids[:test_size]],
        neg_v[neg_eids[:test_size]],
    )
    train_neg_u, train_neg_v = (
        neg_u[neg_eids[test_size:]],
        neg_v[neg_eids[test_size:]],
    )

    # Remove test edges from original graph
    train_g = deepcopy(data)
    train_g.remove_edges(eids[:test_size]) # Remove positive edges from the testing set from the network

    train_pos_g = dgl.graph((train_pos_u, train_pos_v), num_nodes=data.num_nodes())
    train_neg_g = dgl.graph((train_neg_u, train_neg_v), num_nodes=data.num_nodes())

    test_pos_g = dgl.graph((test_pos_u, test_pos_v), num_nodes=data.num_nodes())
    test_neg_g = dgl.graph((test_neg_u, test_neg_v), num_nodes=data.num_nodes())

    return train_g, train_pos_g, train_neg_g, test_pos_g, test_neg_g

def compute_loss(pos_score, neg_score):
    scores = torch.cat([pos_score, neg_score])
    labels = torch.cat(
        [torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])]
    )
    return F.binary_cross_entropy_with_logits(scores, labels)


def compute_auc(pos_score, neg_score):
    scores = torch.cat([pos_score, neg_score]).numpy()
    labels = torch.cat(
        [torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])]
    ).numpy()
    return roc_auc_score(labels, scores)

In [4]:
class GraphSAGE(torch.nn.Module):
    def __init__(self, in_feats, h_feats):
        super(GraphSAGE, self).__init__()
        self.conv1 = SAGEConv(in_feats, h_feats, "mean")
        self.conv2 = SAGEConv(h_feats, h_feats, "mean")

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h
    

class DotPredictor(torch.nn.Module):
    def forward(self, g, h):
        with g.local_scope():
            g.ndata["h"] = h
            # Compute a new edge feature named 'score' by a dot-product between the
            # source node feature 'h' and destination node feature 'h'.
            g.apply_edges(fn.u_dot_v("h", "h", "score"))
            # u_dot_v returns a 1-element vector for each edge so you need to squeeze it.
            return g.edata["score"][:, 0]

In [5]:
def engineer_features(G):
    # TODO Work on getting this to be more feature agnostic - i.e. take the join of all this stuff and null if not present
    # Also need a stored one-hot 

    # Change type to two features, is_student, and is_org
    G_eng = deepcopy(G)
    _type = np.asarray(list(nx.get_node_attributes(G_eng, 'type').items()))
    is_student = np.asarray(_type[:,1] == 'student', dtype='float32')
    # commitment_limit = list(nx.get_node_attributes(G, 'commitment_limit').values())

    X = np.column_stack([is_student, 1-is_student])
    nx.set_node_attributes(G_eng, dict(zip(_type[:,0], X)), 'X')
    nx.set_node_attributes(G_eng, dict(zip(_type[:,0], is_student)), 'class')

    # TODO Add major in as one-hot

    # TODO Add Year in as one-hot


    return G_eng

In [6]:
G = synthetic.synthesize_graph()


In [7]:
G_eng = engineer_features(G)

In [8]:
G = dgl.from_networkx(G_eng, node_attrs=['X', 'class']) # TODO Investigate the slowness here

c:\Users\Jackson Paull\.conda\envs\sd\Lib\site-packages\dgl\backend\pytorch\tensor.py:53: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\b\abs_abjetg6_iu\croot\pytorch_1686932924616\work\torch\csrc\utils\tensor_new.cpp:248.)
  return th.as_tensor(data, dtype=dtype)


In [9]:
train_g, train_pos_g, train_neg_g, test_pos_g, test_neg_g = create_train_test_split_edge(G)

model = GraphSAGE(train_g.ndata["X"].shape[1], 32)
pred = DotPredictor()
optimizer = torch.optim.Adam(
    itertools.chain(model.parameters(), pred.parameters()), lr=0.01
)

In [10]:
# ----------- 4. training -------------------------------- #
all_logits = []
for e in range(1001):
    # forward
    h = model(train_g, train_g.ndata["X"])
    pos_score = pred(train_pos_g, h)
    neg_score = pred(train_neg_g, h)
    loss = compute_loss(pos_score, neg_score)

    # backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if e % 5 == 0:
        print("In epoch {}, loss: {}".format(e, loss))

    # ----------- 5. check results ------------------------ #
    if e % 100 == 0:
        with torch.no_grad():
            pos_score = pred(test_pos_g, h)
            neg_score = pred(test_neg_g, h)
            print("AUC", compute_auc(pos_score, neg_score))

In epoch 0, loss: 7.5411376953125
AUC 0.0309271694214876
In epoch 5, loss: 0.952330470085144
In epoch 10, loss: 0.8892170786857605
In epoch 15, loss: 0.8138831853866577
In epoch 20, loss: 0.652487576007843
In epoch 25, loss: 0.5909802317619324
In epoch 30, loss: 0.5592179894447327
In epoch 35, loss: 0.5456817150115967
In epoch 40, loss: 0.5337920188903809
In epoch 45, loss: 0.5225583910942078
In epoch 50, loss: 0.517717182636261
In epoch 55, loss: 0.5163367390632629
In epoch 60, loss: 0.5148406028747559
In epoch 65, loss: 0.5130857229232788
In epoch 70, loss: 0.5115718841552734
In epoch 75, loss: 0.510759711265564
In epoch 80, loss: 0.5102964639663696
In epoch 85, loss: 0.5097317695617676
In epoch 90, loss: 0.5091122388839722
In epoch 95, loss: 0.5086277723312378
In epoch 100, loss: 0.5081608891487122
AUC 0.9214230371900826
In epoch 105, loss: 0.507703423500061
In epoch 110, loss: 0.5072580575942993
In epoch 115, loss: 0.5068168640136719
In epoch 120, loss: 0.5063924193382263
In epoch 

In [11]:
embeddings = model(train_g, train_g.ndata['X'])

In [60]:
def invert_graph(g, copy_data=True, separate_classes=True):
    
    # Create negative adj mtx
    u, v = g.edges()
    u, v = u.numpy(), v.numpy()
    edge_index = np.array((u, v))
    adj = coo_matrix((np.ones(g.num_edges()), edge_index))
    adj_neg = 1 - adj.todense() - np.eye(g.num_nodes())
    neg_u, neg_v = np.where(adj_neg != 0)

    # Invert the graph

    inv_g = dgl.graph((neg_u, neg_v), num_nodes=g.num_nodes())
    if copy_data:
        for k in g.ndata:
            inv_g.ndata[k] = g.ndata[k]

    # Find and remove all edges between the same class
    if separate_classes:
        with inv_g.local_scope():
            inv_g.apply_edges(lambda edges: {'diff_class' : edges.src['class'] != edges.dst['class']})
            sep = inv_g.edata['diff_class'].numpy()
        inv_g = dgl.remove_edges(inv_g, np.where(~sep)[0])

    return inv_g


def calc_scores(g, model):
    
    with g.local_scope():
        g.ndata["h"] = model(g, g.ndata['X'])
        # TODO replace this with cosine sim
        g.apply_edges(fn.u_dot_v("h", "h", "score"))
        g.apply_edges(lambda edges: {'diff_class' : edges.src['class'] != edges.dst['class']})
        scores = g.edata["score"][:, 0].detach().numpy()
        class_mask = g.edata['diff_class'].numpy()

        return np.column_stack((scores, class_mask))

def output_pipeline(graph: dgl.DGLGraph, 
                    model, 
                    k: int=5, 
                    threshold: float=0.5,  
                    mode: str='topK',
                    invert=True):
    if mode.lower() not in ['topk', 'threshold', 'all']:
        raise ValueError('Mode must be either \'topK\' or \'threshold\' or \'all\'')


    # Create an inverse of the current graph
    # This way we only generate prediction scores for nodes which aren't connected yet
    if invert:
        g = invert_graph(graph)
    else:
        g = deepcopy(graph)

    u, v = g.edges()
    u, v = u.numpy(), v.numpy()
    # eids = np.arange(g.num_edges())
    edges = np.column_stack((u, v))

    scores = calc_scores(g, model)

    # Select only the edges which the class of nodes are different
    mask = np.where(scores[:,1])
    scores = scores[mask][:,0]
    edges = edges[mask]

    order = scores.argsort()[::-1] # Sort descending by score

    scores = scores[order]
    edges = edges[order]

    # if mode is top k, take top k scores
    ret = np.column_stack((edges, scores))
    if mode.lower() == 'topk':
        ret = ret[:k]
        return ret
    if mode.lower() == 'threshold':
        thresh = np.where(ret[:,2] > threshold)
        ret = ret[thresh]
        return ret
    
    # Must be all
    return ret

def node_output_pipelne(graph, node_id, model, k=5, threshold=0.5, mode='topK'):
    # Take the subgraph os stuff only consider node 'node_name'ArithmeticError
    # Pass through output_pipeline
    if mode.lower() not in ['topk', 'threshold', 'all']:
        raise ValueError('Mode must be either \'topK\' or \'threshold\' or \'all\'')

    # TODO This needs debugging - create a subgraph with node 'node_id' and all nodes of different class its not connected to already
    g = invert_graph(graph)
    neighborhood = np.concatenate((g.in_edges(node_id)[0].numpy(), [node_id]))
    sg = g.subgraph(neighborhood)
    ret = output_pipeline(sg, model, mode='all', invert=False)

    # Map old node_id to sg node_id
    nids = sg.ndata[dgl.NID].numpy()
    ret[:,0:2] = nids[ret[:,0:2].astype('int')]

    ret = ret[np.where(ret[:,0] == node_id)] # Do I need this? Maybe

    if mode.lower() == 'topk':
        ret = ret[:k]
        return ret
    if mode.lower() == 'threshold':
        thresh = np.where(ret[:,2] > threshold)
        ret = ret[thresh]
        return ret
    
    # Must be all, return everything
    return ret
    

def format_output(output):
    formatted = {}

    for n in output[:,0]:
        if n not in formatted.keys():
            formatted[int(n)] = {}

    for s in output:
        formatted[int(s[0])][int(s[1])] = s[2]

    return json.dumps(formatted)


        

In [62]:
format_output(node_output_pipelne(G, 15, model))

'{"15": {"13": 2.0949490070343018, "12": 2.0949490070343018, "0": 2.0949490070343018, "1": 2.0949490070343018, "2": 2.0949490070343018}}'